In [1]:
import os
import logging
import tensorflow as tf
from tensorflow.keras import layers, models


2025-06-16 10:51:27.878202: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-16 10:51:27.884963: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-16 10:51:27.903274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750060287.933359  146808 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750060287.942543  146808 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750060287.964842  146808 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# Set up logging
logging.basicConfig(level=logging.INFO)

# Example mock config dictionary (replace with your actual values or load from config files)
model_config_dict = {
    "input_shape": [128],  # Replace with your input feature size
    "optimizer": "adam",
    "classification_loss": "categorical_crossentropy",
    "classification_metric": "accuracy",
    "save_dir": "artifacts/models/base_model"
}

# Ensure model save directory exists
os.makedirs(model_config_dict["save_dir"], exist_ok=True)


In [3]:
from dataclasses import dataclass

@dataclass
class ModelConfig:
    input_shape: list
    optimizer: str
    classification_loss: str
    classification_metric: str
    save_dir: str

    @staticmethod
    def from_dict(config_dict):
        return ModelConfig(**config_dict)


In [4]:
class PrepareBaseModel:
    def __init__(self, config: ModelConfig):
        self.config = config
        self.model = None
        logging.info("PrepareBaseModel initialized.")

    def build_gas_classification_model(self, input_shape):
        inputs = layers.Input(shape=input_shape)
        x = layers.Dense(128, activation='relu')(inputs)
        x = layers.Dropout(0.3)(x)
        x = layers.Dense(64, activation='relu')(x)
        x = layers.Dropout(0.3)(x)
        x = layers.Dense(32, activation='relu')(x)
        outputs = layers.Dense(6, activation='softmax', name='gas_classification')(x)

        model = models.Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=self.config.optimizer,
                      loss=self.config.classification_loss,
                      metrics=[self.config.classification_metric])
        return model

    def save_model(self, path: str, model: tf.keras.Model):
        if os.path.isdir(path):
            path = os.path.join(path, "gas_classification_model.keras")
        model.save(path)
        logging.info(f"Model saved at: {path}")
        return path

    def get_base_model(self):
        input_shape = tuple(self.config.input_shape)
        self.model = self.build_gas_classification_model(input_shape)
        save_path = self.save_model(self.config.save_dir, self.model)
        return self.model, save_path


In [5]:
# Create a config instance
model_config = ModelConfig.from_dict(model_config_dict)

# Instantiate and run model builder
model_preparer = PrepareBaseModel(config=model_config)
model, model_path = model_preparer.get_base_model()


INFO:root:PrepareBaseModel initialized.
2025-06-16 10:51:32.221664: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
INFO:root:Model saved at: artifacts/models/base_model/gas_classification_model.keras


In [6]:
# Visual check of model architecture
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gas_classification (Dense)      │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,046 (105.65 KB)

 Trainable params: 27,046 (105.65 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Reload the saved model to verify it's saved correctly
loaded_model = tf.keras.models.load_model(model_path)
loaded_model.summary()  # Should match the original model


/home/moraa-ontita/Documents/Machine-learning/Gas_Sensor/.venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gas_classification (Dense)      │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 81,140 (316.96 KB)

 Trainable params: 27,046 (105.65 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 54,094 (211.31 KB)